In [1]:
import numpy as np

opcodeInput=np.loadtxt("input_day21.txt",delimiter=",",dtype=np.int64)

class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [7]:
class Computer:
    def __init__(self, opcodeMgr):
        self.opcode = opcodeMgr
        self.outputList=[]
        self.input=0
        
    def clearOutput(self):
        self.outputList=[]
    
    def visualize(self):
        outputStr=''
        for elem in self.outputList:
            outputStr += chr(elem)
        return outputStr
    
    def printOutputList(self):
        outputStr=''
        for elem in self.outputList:
            outputStr += chr(elem)
        print(outputStr)
    
    def run(self, idx):
    
        while True:

            decoded = decodeCommand(self.opcode[idx])
            
            #if len(self.outputList) >= 2 and self.outputList[-2] == -1 and self.outputList[-1] == 0:
            #    print(idx, decoded)

            if decoded[0] == 99:
                print("Finished!")
                return 0
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                #if arg1 + arg2 in [98,59,85]: print(str(arg1) + " + " + str(arg2) + " = " + str(arg1+arg2))
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                #if arg1 * arg2 in [98,59,85]: print(str(arg1) + " * " + str(arg2) + " = " + str(arg1*arg2))
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                # break here and wait for updated input from extern
                #print("Waiting for input")
                yield 1
                var = self.input
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                #print("Output: ", arg1)
                self.outputList.append(arg1)
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [17]:
opcodeMgr=OpCodeManager(opcodeInput.copy())
computer = Computer(opcodeMgr)

#task 1
sequence=["NOT A J", "NOT B T", "OR T J", "NOT C T", "OR T J", "AND D J","WALK"]

#task2
sequence=["NOT A J", "NOT B T", "OR T J", "NOT C T", "OR T J", "AND D J", "NOT J T", "NOT T T", "AND E J ", "AND H T", "OR T J", "RUN"]
#(!A | !B | !C) & D & (E | H) = (J & E) | ( J & H)

inputSequence=''
for i in sequence:
    inputSequence += i + "\n"

print(inputSequence)

for i,step in enumerate(computer.run(0)):
    #print(i,computer.outputList)
    if i == 0:
        print(computer.visualize())
        computer.clearOutput()
    if len(computer.outputList) > 0:
        computer.printOutputList()
        computer.clearOutput()
    if i < len(inputSequence):
        computer.input = ord(inputSequence[i])
#print(computer.visualize())
lastOutput = computer.outputList[-1]
if lastOutput == 10:
    print(computer.visualize())
else:
    print(lastOutput)

NOT A J
NOT B T
OR T J
NOT C T
OR T J
AND D J
NOT J T
NOT T T
AND E J 
AND H T
OR T J
RUN

Input instructions:

Finished!
1140612950
